# Tools involved

## LangSmith
LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!

## GitHub
GitHub is a platform for version control and collaboration, allowing developers to manage and share code repositories. It integrates with Git, a distributed version control system, to track changes, manage branches, and collaborate on software projects. GitHub also provides features like issue tracking, pull requests, and project management tools to streamline development workflows.

## Things I installed
```pip install langchain```

```pip install -U langsmith```

```pip install -qU langchain-openai```

# Environment variables
instead of putting the API keys in the notebook, it's best practice to not share or expose those variables.  In the tutorial, it mentions you can have them imported directly in the notebook but I chose to go with a .env file.  I'm also going to add it to the '.gitignore' file.

## Summary of Steps
1. Create a .env file with your environment variables.
2. Install the python-dotenv package.
3. Load the .env file in your Python script or notebook using load_dotenv().
4. Access the environment variables with os.getenv().

By following these steps, you can securely manage and use environment variables from a .env file in your Python projects. Let me know if you have any further questions!

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Now we can access the variables as usual
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", openai_api_key = openai_api_key)

In [20]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e1f35c0-31dc-4780-9628-351101722f77-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

# Explanation of the Above Cell
We're priming the AI through the SystemMessage, to translate whatever the human says, through the HumanMessage, to Italian.  We're then invoking it
## Langchain_core.messages
### HumaMessage
[Docs](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)

Message from a human.

HumanMessages are messages that are passed in from a human to the model.

### SystemMessage
[Docs](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.system.SystemMessage.html)

Message for priming AI behavior.  

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
# This cell shows that we can take the message and use the parser to only share the 
# result.  Rather than sharing all of the other metadata as well.
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

In [15]:
# This cell goes through the "chain" concept.  The way I understand it, we
# create a chain and then can invoke the message in the same way moving forward.
# Since we set the parser to StrOutputParser(), it just shares the string.
chain = model | parser
chain.invoke(messages)

'Ciao!'

In [16]:
# Now we're creating a PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# This is a string that will be the system message.  We can input different
# languages by having it set to {language}
system_template = "Translate the following into {language}:"

# This prompt template will get the user input and apply it to {text}
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [24]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [27]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [35]:
chain = prompt_template | model | parser

In [36]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

SyntaxError: invalid syntax (2065073899.py, line 1)